<a href="https://www.kaggle.com/code/rohithbehera/whichone?scriptVersionId=133334719" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install timm
import numpy as np
import pandas as pd
import cv2
import torch 
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import timm
import torch.nn as nn
from tqdm import tqdm

In [2]:
data_dir = '/kaggle/input/hotelid-2022-train-images-256x256/'
train_dir = '/kaggle/input/hotelid-2022-train-images-256x256/images/'
OUTPUT_FOLDER = '/kaggle/output/'
df = pd.read_csv('/kaggle/input/hotelid-2022-train-images-256x256/train.csv')

In [3]:
class Customdataset:
    def __init__(self,data,image_path):
        self.data = data
        self.image_path = image_path
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        item = self.data.iloc[idx]
        item_path = self.image_path+item["image_id"]
        image = np.array(Image.open(item_path)).astype(np.uint8)
        return {
            "image":image,
            "target":item["hotel_id_code"]
        }

In [4]:
df["hotel_id_code"] = df["hotel_id"].astype('category').cat.codes.values.astype(np.int64)

In [5]:
hotel_id_code_df = df.drop(columns=["image_id"]).drop_duplicates().reset_index(drop=True)

In [6]:
data = Customdataset(df,train_dir)

In [7]:
class EmbeddingModel(nn.Module):
    def __init__(self,number_of_classes,embedding_size,backbone_name):
        super(EmbeddingModel,self).__init__()
        self.backbone = timm.create_model(backbone_name,num_classes=number_of_classes,pretrained=True)
        output_nodes = self.backbone.get_classifier().in_features
        self.backbone.classifier = nn.Identity()
        self.embedding = nn.Linear(3116,embedding_size)
        self.classifier = nn.Linear(embedding_size,number_of_classes)
    def embed_and_classify(self, x):
        x = self.forward(x)
        return x, self.classifier(x)
    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        x = self.embedding(x)
        return x   

In [8]:
def step(args,model,loader,optimizer,criterion,scheduler,epoch):
    losses = []
    target_all = []
    output_all = []
    model.train()
    item = tqdm(loader)
    for i , data in enumerate(item):
        optimizer.zero_grad()
        inputs = data['image'].to(args.device)
        inputs = inputs.permute(0, 3, 1, 2)
        inputs = inputs.float()
        targets = data['target'].to(args.device)
        embed , output = model.embed_and_classify(inputs)
        loss = criterion(output,targets)
        loss.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()
        losses.append(loss.item())
        target_all.extend(targets.cpu().numpy())
        output_all.extend(torch.sigmoid(output).detach().cpu().numpy())
    net_loss = np.mean(losses)
    score = np.mean(target_all == np.argmax(output_all , axis = 1))
    print(f'at {epoch} the loss , score are {net_loss} , {score}')
    return net_loss , score

In [9]:
def save_checkpoint(model,scheduler,optimizer,start,model_name,loss,score):
     checkpoint = {"epoch": epoch,
                  "model": model.state_dict(),
                  "scheduler": scheduler.state_dict(),
                  "optimizer": optimizer.state_dict(),
                  "loss": loss,
                  "score": score,
                  }
     torch.save(checkpoint, f"{OUTPUT_FOLDER}checkpoint-{name}.pt")

In [10]:
def train(args,df):
    model_name = f"fine tuned {args.backbone}"
    epochs = args.epochs
    start = 1
    criterion = nn.CrossEntropyLoss()
    model = EmbeddingModel(args.n_classes,args.embedding_size,args.backbone)
    model = model.to(args.device)
    print(model)
    dataset = Customdataset(df,train_dir)
    loader = DataLoader(dataset,num_workers=args.workers,batch_size=args.batch_size,shuffle=True, drop_last=True)
    optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
                    optimizer,
                    max_lr=args.lr,
                    epochs=args.epochs,
                    steps_per_epoch=len(loader),
                    div_factor=10,
                    final_div_factor=1,
                    pct_start=0.1,
                    anneal_strategy="cos",
                )
    for start in range(1,epochs+1):
        loss , score = step(args,model,loader,optimizer,criterion,scheduler,start)
        if(start == epochs):
            save_checkpoint(model,scheduler,optimizer,start,model_name,loss,score)   

In [ ]:
%%time 

class args:
    epochs = 30
    lr = 1e-3
    batch_size = 64
    workers = 2
    embedding_size = 512
    backbone = "resnet34"
    n_classes = df["hotel_id_code"].nunique()
    device = ('cuda' if torch.cuda.is_available() else 'cpu')

train(args, df)
    

EmbeddingModel(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), paddi

100%|██████████| 698/698 [11:02<00:00,  1.05it/s]


at 1 the loss , score are 7.00512716968285 , 0.0914890759312321


100%|██████████| 698/698 [10:56<00:00,  1.06it/s]


at 2 the loss , score are 5.669170447270303 , 0.1748522564469914


...

100%|██████████| 698/698 [10:56<00:00,  1.06it/s]


at 10 the loss , score are 0.2926006351082578 , 0.06527578796561605


100%|██████████| 698/698 [10:56<00:00,  1.06it/s]


at 11 the loss , score are 0.24842851477165448 , 0.05401593839541547


 48%|████▊     | 336/698 [05:16<05:42,  1.06it/s]